A number of geographic calculations are made on neighborhoods, often 3x3 (and sometimes larger) around a central pixel. For example, planar slope is calculated on a 3x3 neighborhood, as is the Topographic Roughness Index.  We aim to expand the number of these filters in the neilpy.filters module.

These filters are generally aimed to be used as an ndi.filters.generic_filter.  This method of processing is generally not recommended for large datasets, and is intended to be used for rapid prototyping, testing, and for pedagogical purposes.  You will find these filters much slower than equivalent optimized routines in neilpy (if they exist).

In [ ]:
import neilpy
import rasterio
import numpy as np
import scipy.ndimage as ndi
from skimage.util import apply_parallel
import matplotlib.pyplot as plt

# Load some sample data

We'll use an example DEM of [Mt. Washington](https://en.wikipedia.org/wiki/Mount_Washington) extracted from the National Map, and projected to 10 meter resolution in Web Mercator, but this could be any data you like.  Keep in mind that we make no attempt to correct for differences between [vertical and horizonal map units](https://www.esri.com/arcgis-blog/products/product/imagery/setting-the-z-factor-parameter-correctly/), so you'll need to handle that yourself.  (Neilpy does have a z-factor calculator available for your use.)

In [ ]:
Z, metadata = neilpy.imread('https://github.com/thomaspingel/geodata/raw/master/terrain/mt_washington_10m.tif')
cellsize = metadata['cellsize']
print(metadata)

In [ ]:
plt.imshow(Z,cmap='terrain')

## ESRI Local Planar Slope

ESRI. 2017. <a href="http://desktop.arcgis.com/en/arcmap/10.5/tools/spatial-analyst-toolbox/how-slope-works.htm">How Slope Works</a>.

In [ ]:
from neilpy.filters import esri_planar_slope

In [ ]:
S = ndi.filters.generic_filter(Z,esri_planar_slope,size=3,mode='nearest',extra_keywords={'cellsize':cellsize,'degrees':True})

In [ ]:
plt.figure(figsize=((15,3)))
plt.subplot(131)
plt.imshow(Z,cmap='terrain')
plt.title('elevation')
plt.subplot(132)
plt.imshow(S,cmap='jet',vmin=0,vmax=np.percentile(S,99))
plt.title('slope (degrees)')
plt.subplot(133)
plt.hist(np.random.choice(S.ravel(),10000),bins=50,density=1)
plt.xlabel('slope (degrees)')
plt.ylabel('% share')
plt.show()

# ESRI Curvature

https://www.usna.edu/Users/oceano/pguth/md_help/html/geomorph_curvature.htm
<BR>https://www.esri.com/arcgis-blog/products/product/imagery/understanding-curvature-rasters/
<BR>http://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/how-curvature-works.htm

In [ ]:
from neilpy.filters import esri_curvature

In [ ]:
C = ndi.filters.generic_filter(Z,esri_curvature,size=3,mode='nearest',extra_keywords={'cellsize':cellsize})
C_profile = ndi.filters.generic_filter(Z,esri_curvature,size=3,mode='nearest',extra_keywords={'cellsize':cellsize,'kind':'profile'})
C_plan = ndi.filters.generic_filter(Z,esri_curvature,size=3,mode='nearest',extra_keywords={'cellsize':cellsize,'kind':'plan'})

In [ ]:
plt.figure(figsize=((15,10)))
plt.imshow(C,cmap='bwr',vmin=-.5,vmax=.5)

In [ ]:
plt.figure(figsize=((15,10)))
plt.imshow(C_profile,cmap='bwr',vmin=-.5,vmax=.5)

In [ ]:
plt.figure(figsize=((15,10)))
plt.imshow(C_plan,cmap='bwr',vmin=-.5,vmax=.5)

## Terrain Ruggedness Index

Riley et al. 1999. <a href="http://download.osgeo.org/qgis/doc/reference-docs/Terrain_Ruggedness_Index.pdf">A Terrain Ruggedness Index that Quantifies Topographic Heterogeneity</a>.

In [ ]:
from neilpy.filters import terrain_ruggedness

In [ ]:
TRI = ndi.filters.generic_filter(Z,terrain_ruggedness,size=3)

In [ ]:
plt.figure(figsize=((15,3)))
plt.subplot(131)
plt.imshow(Z,cmap='terrain',vmin=-500,vmax=2000)
plt.title('Elevation')
plt.subplot(132)
plt.imshow(TRI,cmap='jet',vmin=0,vmax=np.percentile(TRI,99))
plt.title('Terrain Ruggedness')
plt.subplot(133)
plt.hist(np.random.choice(TRI.ravel(),10000),bins=50,density=1)
plt.xlabel('Terrain Ruggedness (m)')
plt.ylabel('% share')
plt.show()

# Openness
Yokoyama et al. 2003. <A target="_blank" href="https://pdfs.semanticscholar.org/c3d9/a561fdb9e8c34a2b79152aea72b46090bb2e.pdf">Visualizing Topography by Openness: A New Application of Image Processing to Digital Elevation Models</a>

In [ ]:
from neilpy.filters import openness_filter

In [ ]:
lookup_pixels = 20
O = ndi.filters.generic_filter(Z,openness_filter,size=2*lookup_pixels+1,extra_keywords={'cellsize':cellsize})

In [ ]:
plt.figure(figsize=((15,3)))
plt.subplot(131)
plt.imshow(Z,cmap='terrain',vmin=-500,vmax=2000)
plt.title('Elevation')
plt.subplot(132)
plt.imshow(O,cmap='bone')
plt.title('Positive Openness')
plt.subplot(133)
plt.hist(np.random.choice(O.ravel(),10000),bins=50,density=1)
plt.xlabel('Positive Openness (deg)')
plt.ylabel('% share')
plt.show()